In [1]:
import sys
sys.path.append("/Users/sarathrnair/Projects/tf-transformers/src/")

In [2]:
import tensorflow as tf
from unet import UnetModel

/Users/sarathrnair/miniforge3/envs/venv_tf_mac/lib/python3.9/site-packages/jax/_src/lib/__init__.py:34: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [3]:
out_channels = 64

channel_mult = [1, 2, 3, 4]
num_res_blocks = 3
time_emb = 128
text_emb = 768
input_channels = 3

unet = UnetModel(
                text_embedding_dimension=None, # Make sure output of text encoder matches this
                time_embedding_dimension=time_emb, # This should be same in BaseDiffusion model
                out_channels=out_channels, 
                channel_mult = channel_mult,
                input_channels=input_channels,
                num_res_blocks = num_res_blocks,
                attention_resolutions=[32, 16, 8],
                cross_attention_resolutions=[32, 16, 8],
                use_scale_shift_norm=True,
    image_height = 32,
    image_width = 32

                )



Metal device set to: Apple M1
Down H  (None, 32, 32, 64)
Down H  (None, 16, 16, 64)
Down H  (None, 8, 8, 128)
Down H  (None, 4, 4, 384)
Down H  (None, 4, 4, 1536)

Middle  (None, 4, 4, 1536)
P  (None, 4, 4, 1536)
Cu  (None, 4, 4, 1536)
P (None, 4, 4, 1536) --> Cu (None, 4, 4, 1536) --> Co (None, 4, 4, 3072) --> Out (None, 4, 4, 1536)

P  (None, 4, 4, 384)
Cu  (None, 4, 4, 1536)
P (None, 4, 4, 384) --> Cu (None, 4, 4, 1536) --> Co (None, 4, 4, 1920) --> Out (None, 8, 8, 384)

P  (None, 8, 8, 128)
Cu  (None, 8, 8, 384)
P (None, 8, 8, 128) --> Cu (None, 8, 8, 384) --> Co (None, 8, 8, 512) --> Out (None, 16, 16, 128)

P  (None, 16, 16, 64)
Cu  (None, 16, 16, 128)
P (None, 16, 16, 64) --> Cu (None, 16, 16, 128) --> Co (None, 16, 16, 192) --> Out (None, 32, 32, 64)



In [4]:
unet.input

{'input_pixels': <KerasTensor: shape=(None, 32, 32, 3) dtype=float32 (created by layer 'input_pixels')>,
 'time_steps': <KerasTensor: shape=(1, None) dtype=int32 (created by layer 'time_steps')>}

In [11]:
l = tf.keras.layers.Conv2DTranspose(384, kernel_size=3, strides=2, padding='SAME')

In [7]:
images = tf.random.uniform((10, 32, 32, 3))
time_steps = tf.random.uniform(minval=0, maxval=11, shape=(10, 1))

In [8]:
model_outputs = unet({'input_pixels': images, 'time_steps': time_steps})

Down H  (10, 32, 32, 64)
Down H  (10, 16, 16, 64)
Down H  (10, 8, 8, 128)
Down H  (10, 4, 4, 384)
Down H  (10, 4, 4, 1536)

Middle  (10, 4, 4, 1536)
P  (10, 4, 4, 1536)
Cu  (10, 4, 4, 1536)
P (10, 4, 4, 1536) --> Cu (10, 4, 4, 1536) --> Co (10, 4, 4, 3072) --> Out (10, 4, 4, 1536)

P  (10, 4, 4, 384)
Cu  (10, 4, 4, 1536)
P (10, 4, 4, 384) --> Cu (10, 4, 4, 1536) --> Co (10, 4, 4, 1920) --> Out (10, 8, 8, 384)

P  (10, 8, 8, 128)
Cu  (10, 8, 8, 384)
P (10, 8, 8, 128) --> Cu (10, 8, 8, 384) --> Co (10, 8, 8, 512) --> Out (10, 16, 16, 128)

P  (10, 16, 16, 64)
Cu  (10, 16, 16, 128)
P (10, 16, 16, 64) --> Cu (10, 16, 16, 128) --> Co (10, 16, 16, 192) --> Out (10, 32, 32, 64)



In [9]:
model_outputs['xpred']

<tf.Tensor: shape=(10, 32, 32, 3), dtype=float32, numpy=
array([[[[-9.54634100e-02,  5.19511044e-01,  5.70660472e-01],
         [ 3.60385299e-01, -5.64366877e-01,  6.27293468e-01],
         [-7.50500783e-02,  5.38649142e-01, -6.46964729e-01],
         ...,
         [-8.52673352e-02,  2.21862987e-01,  9.19163942e-01],
         [ 1.31796673e-01, -1.36558965e-01,  3.47679585e-01],
         [-3.16826105e-02,  7.94653073e-02, -1.49725527e-02]],

        [[ 5.55644572e-01, -5.43783784e-01, -7.70685494e-01],
         [ 6.32973492e-01,  8.60764980e-01,  1.88137746e+00],
         [ 4.65020001e-01, -9.42035139e-01,  1.58872044e+00],
         ...,
         [-8.89064908e-01,  1.12609431e-01,  1.49440300e+00],
         [ 7.60103345e-01, -4.27213758e-01, -6.18870780e-02],
         [ 8.49362612e-01,  1.79017365e-01, -1.98813990e-01]],

        [[-1.69070140e-01, -7.46081352e-01,  1.12507805e-01],
         [-4.08965588e-01, -1.47740698e+00,  5.01009941e-01],
         [-1.43604767e+00, -4.08992857e-01,